In [3]:
%pip install taker

Note: you may need to restart the kernel to use updated packages.


In [4]:
import circuitsvis as cv
import numpy as np
import torch
from taker import Model
from taker.hooks import HookConfig
from datetime import datetime
import json
from os import listdir
from os.path import exists

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import torch

/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"../results/{current_time}_agnes_story.jsonl"
print(filename)

../results/2024-08-01_15-22-27_agnes_story.jsonl


In [6]:
m = Model("microsoft/phi-3-mini-4k-instruct", dtype="int4")
m.show_details()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.78s/it]


Loaded model 'microsoft/phi-3-mini-4k-instruct' with int4:
- Added 512 hooks across 32 layers


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.
/workspace/SPAR/interp-ab/.venv/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


 - n_layers : 32
 - d_model  : 3072
 - n_heads  : 32
 - d_head   : 96
 - d_mlp    : 8192


In [7]:
prompt    = """Write a short blog post about a recipe and the inspiration behind it.
 Do not include a title.
 Only reveal the dish after the story.
 Start with short story and then move to the recipe.
 To re-iterate, do not include a title."""
# info_gen = m.generate(info_prompt, temperature=0.3, num=300)
info_gen = """
\n Once upon a time, in a quaint little village nestled between rolling hills and verdant fields, there lived an elderly woman named Agnes. Agnes was known for her warm smile and her legendary Sunday dinners that brought the entire neighborhood together. Her recipes were family heirlooms, passed down through generations, with each family adding their own touch to the final dish.

One crisp autumn evening, Agnes was reminiscing about her childhood, and how her grandmother used to gather everyone around the dinner table, sharing stories and laughter. These were the moments that shaped her, the memories that she passed on to her own children and grandchildren.

Inspired by her grandmother's legacy, Agnes decided to create a new dish that would encapsulate the essence of those cherished gatherings. She wanted something that was comforting and nourishing, a dish that could be prepared with love and shared with others. After days of experimentation, she finally created a recipe that she believed truly captured the spirit of her family's Sunday dinners.\n
"""
info_prompt = prompt+info_gen

In [8]:
import sys, os
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [8]:
# if not exists(filename):
#     with open(filename, "w") as f:
#         pass

# neutral_prompt = "\n"
# max_new_tokens = 100
# temperature = 0.2

# with HiddenPrints():
#     for i in range(20):
#         output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
        
#         data = {
#             "temperature": temperature,
#             "max_new_tokens": max_new_tokens,
#             "transplant_layers": None,
#             "layers_type": None,
#             "orig_prompt": info_prompt,
#             "transplant_prompt": neutral_prompt,
#             "output": output[1],
#         }

#         with open(filename, "a") as file:
#             file.write(json.dumps(data) + "\n")
            
# acts = m.get_midlayer_activations(info_prompt)
# orig_token_index = m.get_ids(info_prompt).shape[1] - 1
# new_token_index  = m.get_ids(neutral_prompt).shape[1] - 1        


# [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
# for layer_index in range(0,32):
#     m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index])
#     m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index])
# with HiddenPrints():
#     for i in range(20):
#         output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
        
#         data = {
#             "temperature": temperature,
#             "max_new_tokens": max_new_tokens,
#             "transplant_layers": (0,32),
#             "layers_type": "all",
#             "orig_prompt": info_prompt,
#             "transplant_prompt": neutral_prompt,
#             "output": output[1],
#         }

#         with open(filename, "a") as file:
#             file.write(json.dumps(data) + "\n")
                
                
# [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
# for layer_index in range(0,32):
#     # m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index])
#     m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index])
# with HiddenPrints():
#     for i in range(20):
#         output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
        
#         data = {
#             "temperature": temperature,
#             "max_new_tokens": max_new_tokens,
#             "transplant_layers": (0,32),
#             "layers_type": "attn",
#             "orig_prompt": info_prompt,
#             "transplant_prompt": neutral_prompt,
#             "output": output[1],
#         }

#         with open(filename, "a") as file:
#             file.write(json.dumps(data) + "\n")
                
# [h.reset() for h in m.hooks.neuron_replace.values()] #RESET HOOKS BEFORE TRANSPLANTING NEXT SET OF ACTIVATIONS
# for layer_index in range(0,32):
#     m.hooks.neuron_replace[f"layer_{layer_index}_mlp_pre_out"].add_token(new_token_index, acts["mlp"][0, layer_index, orig_token_index])
#     # m.hooks.neuron_replace[f"layer_{layer_index}_attn_pre_out"].add_token(new_token_index, acts["attn"][0, layer_index, orig_token_index])
# with HiddenPrints():
#     for i in range(20):
#         output = m.generate(neutral_prompt, max_new_tokens, temperature=temperature)
        
#         data = {
#             "temperature": temperature,
#             "max_new_tokens": max_new_tokens,
#             "transplant_layers": (0,32),
#             "layers_type": "mlp",
#             "orig_prompt": info_prompt,
#             "transplant_prompt": neutral_prompt,
#             "output": output[1],
#         }

#         with open(filename, "a") as file:
#             file.write(json.dumps(data) + "\n")

In [9]:
filepath = "../results/2024-08-01_14-43-20_agnes_story.jsonl"
results_df = pd.read_json(filepath, lines=True)
results_df.head()

,temperature,max_new_tokens,transplant_layers,layers_type,orig_prompt,transplant_prompt,output
0,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,\n\n**Instruction 1 (Same Diffemenct):**\n\n Y...
1,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,\n\n**Exercise 1:**\n\nLet's consider a 7-sent...
2,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,"\n\nHere is a modified, more challenging versi..."
3,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,\n\n**Question:**\nHow to enhance memory reten...
4,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,"\n\nExercise 1:\nIn a small town, there is a t..."


In [10]:
# sentenceTransformer = SentenceTransformer("all-MiniLM-L6-v2")
sentenceTransformer = SentenceTransformer("all-mpnet-base-v2")

embeddings = sentenceTransformer.encode(results_df["output"].to_list())
print(embeddings.shape)
results_df["embeddings"] = embeddings.tolist()
results_df.head()

(80, 768)


,temperature,max_new_tokens,transplant_layers,layers_type,orig_prompt,transplant_prompt,output,embeddings
0,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,\n\n**Instruction 1 (Same Diffemenct):**\n\n Y...,"[0.037170153111219406, -0.012575767002999783, ..."
1,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,\n\n**Exercise 1:**\n\nLet's consider a 7-sent...,"[0.03880508616566658, 0.044158194214105606, -0..."
2,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,"\n\nHere is a modified, more challenging versi...","[-0.00243499968200922, 0.0798662006855011, 0.0..."
3,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,\n\n**Question:**\nHow to enhance memory reten...,"[0.002692834008485079, 0.014343745075166225, -..."
4,0.2,100,None,None,Write a short blog post about a recipe and the...,\n,"\n\nExercise 1:\nIn a small town, there is a t...","[-0.00047723331954330206, 0.021705100312829018..."


In [13]:
# carry out pca on embeddings using sklearn
reducer = PCA(n_components=2)
embeddings_reduced = reducer.fit_transform(embeddings)
print(embeddings_reduced.shape)

# visualize the embeddings using plotly
import plotly.express as px

results_df["dim1"] = embeddings_reduced[:, 0]
results_df["dim2"] = embeddings_reduced[:, 1]
# results_df["dim3"] = embeddings_reduced[:, 2]

results_df["truncated_output"] = results_df["output"].str.slice(0,100)

def convert_value(value):
    if value is None:
        return "None"
    else:
        return value
results_df['layers_type_string'] = results_df['layers_type'].apply(convert_value)

fig = px.scatter(
    results_df,
    x="dim1",
    y="dim2",
    hover_data=["layers_type_string"],
    title="PCA of Agnes Story Embeddings by Layer Type",
    color="layers_type_string",
    # symbol_sequence = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'pentagon', 'hexagon', 'octagon', 'star'],

)
fig.update_traces(
    hovertemplate="<br>".join(
        ["Layers Type: %{customdata[0]}"]
    )
)



fig.show()

(80, 2)


In [14]:
import plotly.graph_objects as go
pca_df = pd.DataFrame(embeddings_reduced, columns=['PC1', 'PC2'])
pca_df['layers_type_string'] = results_df['layers_type_string']

mean_pca_df = pca_df.groupby('layers_type_string').mean().reset_index()
print(mean_pca_df.head())

# fig.add_trace(px.scatter(mean_pca_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', 
#                          title='Means of PCA Groups').data[0]):
mean_pca_df['type'] = 'Mean'
pca_df['type'] = 'Original'
combined_df = pd.concat([pca_df, mean_pca_df], ignore_index=True)
# fig = px.scatter(combined_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', title='PCA of Sentence Embeddings')

# Plot mean points
mean_fig = px.scatter(mean_pca_df, x='PC1', y='PC2', color='layers_type_string', title='Means of PCA Groups')
    

mean_fig.show()

  layers_type_string       PC1       PC2
0               None  0.322960 -0.063794
1                all -0.362840  0.139441
2               attn -0.416063 -0.060083
3                mlp  0.455943 -0.015565


In [14]:
fig.data=[]
mean_fig.data = []

In [19]:
# carry out pca on embeddings using sklearn
reducer = PCA(n_components=3)
embeddings_reduced = reducer.fit_transform(embeddings)
print(embeddings_reduced.shape)

results_df["dim1"] = embeddings_reduced[:, 0]
results_df["dim2"] = embeddings_reduced[:, 1]
results_df["dim3"] = embeddings_reduced[:, 2]

results_df["truncated_output"] = results_df["output"].str.slice(0,100)
def convert_value(value):
    if value is None:
        return "None"
    else:
        return f"{value[0]} - {value[1]-1}"
results_df['layers_string'] = results_df['transplant_layers'].apply(convert_value)

fig = px.scatter_3d(
    results_df,
    x="dim1",
    y="dim2",
    z="dim3",
    hover_data=["temperature", "layers_string", "truncated_output"],
    title="PCA of Agnes Story Embeddings",
    color="temperature",
    symbol="layers_string",
    symbol_sequence = ['circle', 'circle-open', 'cross', 'diamond',
            'diamond-open', 'square', 'square-open', 'x'],

)
fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=100, xref="container"))

fig.update_traces(
    hovertemplate="<br>".join(
        ["Temperature: %{customdata[0]}", "Transplanted Layers: %{customdata[1]}", "Output: %{customdata[2]}"]
    )
)



fig.show()

(640, 3)


In [20]:
import plotly.graph_objects as go
pca_df = pd.DataFrame(embeddings_reduced, columns=['PC1', 'PC2','PC3'])
pca_df['temperature'] = results_df['temperature']
pca_df['layers_string'] = results_df['layers_string']

mean_pca_df = pca_df.groupby(['temperature', 'layers_string']).mean().reset_index()
print(mean_pca_df.head())

# fig.add_trace(px.scatter(mean_pca_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', 
#                          title='Means of PCA Groups').data[0]):
mean_pca_df['type'] = 'Mean'
pca_df['type'] = 'Original'
combined_df = pd.concat([pca_df, mean_pca_df], ignore_index=True)
# fig = px.scatter(combined_df, x='PC1', y='PC2', color='temperature', symbol='layers_string', title='PCA of Sentence Embeddings')

# Plot mean points
mean_fig = px.scatter_3d(mean_pca_df, x='PC1', y='PC2', z='PC3', color='temperature', symbol='layers_string', title='Means of PCA Groups', category_orders={"layers_string": ["None", "0 - 31", "0 - 10", "0 - 15", "20 - 31", "15 - 31", "25 - 31", "0 - 5"]}, symbol_sequence = ['circle', 'circle-open', 'cross', 'diamond',
            'diamond-open', 'square', 'square-open', 'x'],)
    
mean_fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=100, xref="container"))


mean_fig.show()


   temperature layers_string       PC1       PC2       PC3
0          0.2        0 - 10  0.246505 -0.096233 -0.059515
1          0.2        0 - 15  0.269950 -0.213564 -0.052817
2          0.2        0 - 31 -0.388800 -0.325690  0.267034
3          0.2         0 - 5 -0.148765 -0.004382 -0.039764
4          0.2       15 - 31  0.214204  0.371588  0.421958
